In [8]:
import pandas as pd
import math
import numpy as np
import sqlite3
from werkzeug.security import generate_password_hash

# from pyDecision.algorithm import moora_method,saw_method
import pyDecision.algorithm as mcdma

ModuleNotFoundError: No module named 'pandas'

In [1]:
pip install pandas

     ---------------------------------------- 10.9/10.9 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 14.9/14.9 MB 478.4 kB/s eta 0:00:00
     ------------------------------------ 499.4/499.4 kB 978.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install math

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


In [ ]:
tahun = 2021
xl = pd.ExcelFile("data/dummies.xlsx")

In [ ]:
xl.sheet_names

['user', 'profil', 'penilaian', 'kriteria', 'skor']

In [ ]:
user = xl.parse(xl.sheet_names[0])
user['password'] = user.apply(lambda x: generate_password_hash(x['password'], method='sha256'), axis=1)
user = user.rename(columns={'uid':'id'})
user.head()

,username,id,password
0,kepsek,u001,sha256$CgNt7P1LOgmPtxiG$e697150b9d17489245eeee...
1,admin,u004,sha256$hXu5yEMGz7NXgx59$ab0fd3c9ffd6a6727c5bbd...


In [ ]:
profil = xl.parse(xl.sheet_names[1])
profil['nuptk'] = profil['nuptk'].fillna(0)
profil['nuptk'] = profil['nuptk'].astype(int).astype(str)
profil['nrg'] = profil['nrg'].fillna(0)
profil['nrg'] = profil['nrg'].astype(int).astype(str)
profil['ttl'] = pd.to_datetime(profil['ttl'])
profil['tmt'] = pd.to_datetime(profil['tmt'])
profil['mulai'] = pd.to_datetime(profil['mulai'])
profil.head()

,uid,nip,karpeg,nuptk,nrg,nama,ttl,gol,tmt,mulai,pendidikan,gender,mata_pelajaran,bidang_yang_diampu
0,u001,198505192015032856,B05017957,2851763664130020,141561293313,"Jasmin, S.Pd.SD",1974-11-30,VIA,1998-03-05,2006-10-15,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Kepala Sekolah
1,u002,198505192015031819,B05017869,2851763664130256,141561293502,"Kutsiyah, S.Pd",1984-04-29,IIID,2007-03-13,2012-01-18,Pendidikan Jasmani,Perempuan,Pendidikan Jasmani dan Kesehatan,Guru Pendidikan Jasmani dan Kesehatan
2,u003,198505192015031387,B05017826,2851763664130231,141561293323,"Sendja Indah, S.Pd",1984-01-28,IIID,2006-09-01,2012-11-05,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Guru Kelas I A dan B
3,u004,198505192015031907,B05017856,0,0,Nama52,1988-10-03,IIIC,2011-08-22,2013-01-26,NaN,Laki-laki,Admin,NaN
4,u005,198505192015032861,B05017963,2851763664130246,141561293388,"Lestari, S.Pd",1987-08-12,IIIC,2011-05-07,2013-11-24,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Guru Kelas III A


In [ ]:
nilai = xl.parse(xl.sheet_names[2])
nilai.head()

,pid,uid,tgl_penilaian,tahun,A01,A02,A03,A04,A05,A06,...,L03,M01,M02,M03,N01,N02,N03,N04,N05,N06
0,p001,u002,2020-12-21,2020,2,0,0,1,2,1,...,1,0,1,0,0,1,1,2,0,1
1,p002,u003,2020-12-21,2020,2,1,2,2,1,1,...,1,1,1,2,2,1,2,0,2,1
2,p003,u005,2020-12-21,2020,2,1,2,1,1,0,...,2,2,2,1,1,1,1,2,2,1
3,p004,u006,2020-12-21,2020,1,1,2,1,2,2,...,2,1,1,2,1,2,1,1,2,2
4,p005,u007,2020-12-21,2020,1,2,0,0,0,1,...,1,1,2,0,1,2,1,1,1,0


In [ ]:
kriteria = xl.parse(xl.sheet_names[3])
kriteria.head()

,indikator_id,kompetensi,indikator
0,A01,Mengenal karakteristik peserta didik,Guru dapat mengidentifikasi karakteristik bela...
1,A02,Mengenal karakteristik peserta didik,Guru memastikan bahwa semua peserta didik mend...
2,A03,Mengenal karakteristik peserta didik,Guru dapat mengatur kelas untuk memberikan kes...
3,A04,Mengenal karakteristik peserta didik,Guru mencoba mengetahui penyebab penyimpangan ...
4,A05,Mengenal karakteristik peserta didik,Guru membantu mengembangkan potensi dan mengat...


In [ ]:
skor = xl.parse(xl.sheet_names[4])
skor.head()

,skor,keterangan
0,0,Tidak ada bukti (Tidak terpenuhi)
1,1,Terpenuhi sebagian
2,2,Seluruhnya terpenuhi


In [ ]:
con = sqlite3.connect("db/data.db",check_same_thread=False)
user.to_sql('user',con=con,if_exists='replace',index=False)
profil.to_sql('profil',con=con,if_exists='replace',index=False)
nilai.to_sql('nilai',con=con,if_exists='replace',index=False)
kriteria.to_sql('kriteria',con=con,if_exists='replace',index=False)
skor.to_sql('skor',con=con,if_exists='replace',index=False)

3

In [ ]:
# def normalization(matrix):
#     # Transpose Decision Matrix
#     matrix = matrix.transpose()
#     row_values = []
#     norm_matrix = []
    
#     for i in range(matrix.shape[0]): # Looping per baris (kriteria)
#         # Menghitung sum tiap x_{ij}^2
#         sum_row = sum([pow(x,2) for x in matrix[i]])
        
#         for j in range(matrix[i].shape[0]): # Looping per kolom (alternatif)
#             # membangi nilai asli x_{ij} dengan hasil akar
#             r_value = matrix[i][j] / math.sqrt(sum_row)
            
#             # Masukkan hasil normalisasi ke list tiap baris
#             row_values.append(r_value)
        
#         #Masukkan hasil normalisasi per baris ke matrix normalisasi
#         norm_matrix.append(row_values)
        
#         #Kosongkan list normalisasi perbaris
#         row_values = []
            
#     # Ubah dalam bentuk numpy array
#     norm_matrix = np.asarray(norm_matrix)
    
#     # Return dalam bentuk transporse agar kembali ke format awal
#     return norm_matrix.transpose()

In [ ]:
# # Implementasi Menghitung Nilai 
# def optimize_value(w_matrix):
#     y_values = []
    
#     for i in range(w_matrix.shape[0]):
#         max_val = []
#         min_val = []
        
#         for j in range(w_matrix[i].shape[0]):
#             max_val.append(w_matrix[i][j])
        
#         y = sum(max_val) - sum(min_val)
#         y_values.append(y)
    
#     return np.asarray(y_values)

In [ ]:
def bobot(nilai):
    if nilai <=25:
        return 1
    elif nilai >25 and nilai <=50:
        return 2
    elif nilai >50 and nilai <=75:
        return 3
    else:
        return 4

In [ ]:
w = [
    1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6),
    1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6),
    1/(14*4), 1/(14*4), 1/(14*4), 1/(14*4), 
    1/(14*11), 1/(14*11), 1/(14*11), 1/(14*11), 1/(14*11), 1/(14*11), 1/(14*11), 1/(14*11), 1/(14*11), 1/(14*11), 1/(14*11), 
    1/(14*7), 1/(14*7), 1/(14*7), 1/(14*7), 1/(14*7), 1/(14*7), 1/(14*7), 
    1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6),
    1/(14*5), 1/(14*5), 1/(14*5), 1/(14*5), 1/(14*5), 
    1/(14*5), 1/(14*5), 1/(14*5), 1/(14*5), 1/(14*5), 
    1/(14*5), 1/(14*5), 1/(14*5), 1/(14*5), 1/(14*5), 
    1/(14*8), 1/(14*8), 1/(14*8), 1/(14*8), 1/(14*8), 1/(14*8), 1/(14*8), 1/(14*8), 
    1/(14*3), 1/(14*3), 1/(14*3), 
    1/(14*3), 1/(14*3), 1/(14*3), 
    1/(14*3), 1/(14*3), 1/(14*3), 
    1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6), 1/(14*6),
]

c = ["max" for i in nilai.columns[4:]]

In [ ]:
def moora(tahun):
    nilai = pd.read_sql_query(
        f"select * from nilai where tahun = {tahun}",
        con=con
    )

    kriteria = pd.read_sql_query(
        "select * from kriteria",
        con=con
    )

    profil = pd.read_sql_query(
        "select * from profil",
        con=con
    )

    df = nilai.iloc[:,:4].copy()
    for m in  kriteria['indikator_id'].str[0].unique():
        df[m] = pd.DataFrame(nilai[[n for n in nilai.columns if n.startswith("A")]].sum(axis=1)*100/(len([n for n in nilai.columns if n.startswith("A")])*2), columns=['A']).apply(lambda x: bobot(x.values[0]), axis=1)
    # return df.iloc[:,:4].join(pd.DataFrame(optimize_value(normalization(df.iloc[:,4:].values)),columns=['skor'])).sort_values(by='skor',ascending=False).set_index('uid').join(profil.set_index('uid')).reset_index(), \
    return df.iloc[:,:4].join(pd.DataFrame(mcdma.moora_method(nilai.iloc[:,4:].values,w,c,graph=False)[:,1],columns=["skor"])).sort_values(by='skor',ascending=False).set_index('uid').join(profil.set_index('uid')).reset_index(), \
        df.rename(columns={n:kriteria[kriteria['indikator_id'].str[0]==n]['kompetensi'].unique()[0] for n in kriteria['indikator_id'].str[0].unique()})

In [ ]:
def topsis(tahun):
    nilai = pd.read_sql_query(
        f"select * from nilai where tahun = {tahun}",
        con=con
    )

    kriteria = pd.read_sql_query(
        "select * from kriteria",
        con=con
    )

    profil = pd.read_sql_query(
        "select * from profil",
        con=con
    )

    df = nilai.iloc[:,:4].copy()
    for m in  kriteria['indikator_id'].str[0].unique():
        df[m] = pd.DataFrame(nilai[[n for n in nilai.columns if n.startswith("A")]].sum(axis=1)*100/(len([n for n in nilai.columns if n.startswith("A")])*2), columns=['A']).apply(lambda x: bobot(x.values[0]), axis=1)
    # return df.iloc[:,:4].join(pd.DataFrame(optimize_value(normalization(df.iloc[:,4:].values)),columns=['skor'])).sort_values(by='skor',ascending=False).set_index('uid').join(profil.set_index('uid')).reset_index(), \
    return df.iloc[:,:4].join(pd.DataFrame(mcdma.topsis_method(nilai.iloc[:,4:].values,w,c,graph=False),columns=["skor"])).sort_values(by='skor',ascending=False).set_index('uid').join(profil.set_index('uid')).reset_index(), \
        df.rename(columns={n:kriteria[kriteria['indikator_id'].str[0]==n]['kompetensi'].unique()[0] for n in kriteria['indikator_id'].str[0].unique()})

In [ ]:
%%time
df1,dx1 = moora(tahun)

a1: 0.194
a2: 0.199
a3: 0.227
a4: 0.21
a5: 0.221
a6: 0.235
a7: 0.223
a8: 0.226
a9: 0.229
a10: 0.224
a11: 0.222
a12: 0.232
a13: 0.241
a14: 0.246
a15: 0.229
CPU times: user 95.9 ms, sys: 3.65 ms, total: 99.6 ms
Wall time: 98.8 ms


In [ ]:
%%time
df2,dx2 = topsis(tahun)

a1: 0.46
a2: 0.47
a3: 0.54
a4: 0.52
a5: 0.54
a6: 0.56
a7: 0.53
a8: 0.57
a9: 0.56
a10: 0.53
a11: 0.54
a12: 0.55
a13: 0.56
a14: 0.59
a15: 0.55
CPU times: user 112 ms, sys: 303 µs, total: 113 ms
Wall time: 115 ms
